### Data Extraction and Cleaning script  - CO2
The script extracts the CO2 from files obtained from ftp://ftp.cmdl.noaa.gov/data/greenhouse_gases/co2/flask/surface/

Only Monthly files have been considered.


In [1]:
# Import Dependencies
import csv
import os
import pandas as pd
import numpy as np

In [2]:
filePath = "..\data\_rawdata\CO2\surface\Month"
CountryFile = "..\data\_rawdata\CO2\stations_Country_latest.csv"


In [3]:
fileList = []
# Read the dir and list the name of the files
for root, dirs, files in os.walk(filePath):
    for name in files:
        if(name not in fileList):
            fileList.append(name)

print(f"Data to be extracted from {len(fileList)} files")


Data to be extracted from 95 files


In [4]:
txtData = []
CO2Data = pd.DataFrame()

for i, f in enumerate(fileList):
    with(open(f"{filePath}\{f}", "r")) as txtFile:
        txtData = txtFile.read().splitlines()
    
    # DAta is space delimited format. But not uniform. 
    # Change spaces to , and split field values by commas
    tmpList = [l.replace(" ",",") for l in txtData[int(txtData[0].rsplit(" ", 1)[1]):]]
    tmpList = [l.split(",") for l in tmpList]
    
    # add data to dataframe
    CO2Data = CO2Data.append(tmpList)



#Format the values and correct incorrect split of data    
CO2Data['month'] = CO2Data[2].add(CO2Data[3], fill_value = 0)
CO2Data['month'] = CO2Data['month'].add(CO2Data[4], fill_value = 0)
CO2Data[[5,6]] = CO2Data[[5,6]].apply(pd.to_numeric)
CO2Data['CO2Value'] = CO2Data[5].add(CO2Data[6], fill_value = 0)

# Drop cols not needed
CO2Data = CO2Data.drop([2,3,4,5,6], axis = 1)

# Change column names, and data types
CO2Data.rename(columns = {0 : 'Code',1 : 'year'}, inplace = True)

#convert all number columns to numeric datatype
CO2Data[['CO2Value','year','month']] = CO2Data[['CO2Value','year','month']].apply(pd.to_numeric)


CO2Data.head()

,Code,year,month,CO2Value
0,ABP,2006,10,380.72
1,ABP,2006,11,380.82
2,ABP,2006,12,380.92
3,ABP,2007,1,381.02
4,ABP,2007,2,381.09


In [5]:
print(f"data from {CO2Data['year'].min()} to {CO2Data['year'].max()}")

data from 1968 to 2017


In [6]:
# Remove all data before 1970.
CO2Data = CO2Data.loc[CO2Data['year']>= 1975,:]

print(f"data from {CO2Data['year'].min()} to {CO2Data['year'].max()}")

data from 1975 to 2017


In [7]:
#Reshape data to be same as Temp view
CO2Data_fnl = CO2Data.pivot_table('CO2Value', ['Code','year'], 'month')

CO2Data_fnl = CO2Data_fnl.reset_index()


CO2Data_fnl.rename_axis("", axis = 1, inplace = True)

mnth_names = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov', 12:'Dec'}
CO2Data_fnl.rename(columns = mnth_names, inplace = True)

CO2Data_fnl.head()


,Code,year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,000,1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,349.66,NaN,NaN,NaN
1,000,1988,351.31,352.37,353.04,353.35,353.30,353.05,352.68,352.29,NaN,NaN,NaN,NaN
2,000,1989,354.37,355.10,355.46,355.34,354.93,354.38,353.86,353.47,353.19,353.17,353.46,354.11
3,000,1990,354.88,355.47,355.86,356.00,355.96,355.76,355.39,354.85,354.32,354.08,354.28,354.94
4,000,1991,355.71,356.40,356.87,357.04,356.94,356.64,356.18,355.51,354.88,354.61,354.82,355.46


In [8]:
#Fix Missing Values - with avgs of NAN row
CO2Data_clean = CO2Data_fnl

rowMean = CO2Data_clean[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov', 'Dec']].mean(axis = 1)


CO2Data_clean = CO2Data_clean.apply(lambda r: r.fillna(rowMean[r.index]))

CO2Data_clean = CO2Data_clean.round(decimals = 2)

#Check if NAN exists
print(f"Num of duplicates is {CO2Data_clean.isnull().values.sum()}")

CO2Data_clean.head()

Num of duplicates is 0


,Code,year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,000,1987,349.66,349.66,349.66,349.66,349.66,349.66,349.66,349.66,349.66,349.66,349.66,349.66
1,000,1988,351.31,352.37,353.04,353.35,353.30,353.05,352.68,352.29,352.67,352.67,352.67,352.67
2,000,1989,354.37,355.10,355.46,355.34,354.93,354.38,353.86,353.47,353.19,353.17,353.46,354.11
3,000,1990,354.88,355.47,355.86,356.00,355.96,355.76,355.39,354.85,354.32,354.08,354.28,354.94
4,000,1991,355.71,356.40,356.87,357.04,356.94,356.64,356.18,355.51,354.88,354.61,354.82,355.46


In [9]:
# Lookup Country and Add it to CO2 Data
ctry_stat = pd.read_csv(CountryFile)
ctry_stat.head()


,Code,Name,Country,Latitude,Longitude,Elevation (meters),Time from GMT
0,AAO,"Airborne Aerosol Observatory, Bondville, Illinois",United States,40.050,-88.370,230.0,-6 hours
1,ABP,"Arembepe, Bahia",Brazil,-12.770,-38.170,1.0,-3 hours
2,ABQ,"Albuquerque, New Mexico",United States,35.038,-106.622,1617.0,-7 hours
3,ACG,Alaska Coast Guard,United States,65.000,-165.000,0.0,-8 hours
4,ALT,"Alert, Nunavut",Canada,82.451,-62.507,190.0,-4 hours


In [10]:
# Merge on Station Code for details about the station
CO2Data_Full = pd.merge(CO2Data_clean, ctry_stat, how = 'inner', on = 'Code')

In [11]:
CO2Data_Full.head() 

,Code,year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Name,Country,Latitude,Longitude,Elevation (meters),Time from GMT
0,ABP,2006,380.82,380.82,380.82,380.82,380.82,380.82,380.82,380.82,380.82,380.72,380.82,380.92,"Arembepe, Bahia",Brazil,-12.770,-38.170,1.0,-3 hours
1,ABP,2007,381.02,381.09,380.99,380.89,381.15,381.82,382.26,382.19,382.34,381.53,381.53,381.53,"Arembepe, Bahia",Brazil,-12.770,-38.170,1.0,-3 hours
2,ABP,2008,384.23,384.23,383.02,382.64,382.83,383.83,384.34,384.20,384.77,385.46,385.64,385.56,"Arembepe, Bahia",Brazil,-12.770,-38.170,1.0,-3 hours
3,ABP,2009,384.85,385.28,386.19,385.20,384.39,385.18,385.67,386.08,386.20,386.18,386.59,386.81,"Arembepe, Bahia",Brazil,-12.770,-38.170,1.0,-3 hours
4,ALT,1985,344.06,344.06,344.06,344.06,344.06,349.94,343.98,337.96,339.09,342.85,345.66,348.97,"Alert, Nunavut",Canada,82.451,-62.507,190.0,-4 hours


In [12]:
# Roll up stations to Country

CO2Data_Cntry_GDF = CO2Data_Full[['Code','year','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep',\
                                  'Oct','Nov','Dec','Country']].groupby(['year','Country']).agg(np.mean)

CO2Data_Cntry_GDF.reset_index(inplace= True)

CO2Data_Cntry_GDF = CO2Data_Cntry_GDF.round(decimals = 2)

CO2Data_Cntry_GDF.head()


,year,Country,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1975,American Samoa,330.65,330.65,330.20,329.63,329.66,330.60,330.83,330.91,331.79,331.64,330.88,330.56
1,1975,United States,333.32,333.24,333.22,333.30,333.16,332.18,329.34,327.23,327.44,329.78,331.89,333.15
2,1976,American Samoa,330.52,330.60,330.82,331.17,331.42,331.10,330.98,331.53,331.90,332.17,332.62,332.95
3,1976,United States,332.56,332.80,333.28,334.17,334.15,332.84,331.08,329.01,328.64,329.98,331.74,333.10
4,1977,American Samoa,332.98,332.48,332.21,332.66,332.82,332.56,332.81,332.91,332.62,332.57,333.01,333.68


In [13]:
# Roll up stations to Global level

CO2Data_glb_GDF = CO2Data_Full[['year','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep',\
                                  'Oct','Nov','Dec']].groupby(['year']).agg(np.mean)

CO2Data_glb_GDF.reset_index(inplace= True)

CO2Data_glb_GDF = CO2Data_glb_GDF.round(decimals = 2)

CO2Data_glb_GDF.head()

,year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1975,332.43,332.38,332.21,332.08,331.99,331.65,329.84,328.45,328.89,330.40,331.55,332.29
1,1976,332.26,332.49,332.93,333.74,333.76,332.59,331.06,329.37,329.10,330.30,331.87,333.08
2,1977,333.94,334.30,334.73,335.59,335.93,334.96,333.34,331.67,330.96,331.78,333.53,334.95
3,1978,335.66,336.13,336.54,336.88,337.07,336.66,334.92,333.05,332.72,333.68,335.20,336.13
4,1979,337.40,337.90,338.65,339.11,339.25,338.33,336.09,334.40,334.50,335.80,337.08,337.80


In [16]:
#Rename Columns to Standard maintained by other files
CO2Data_Full.rename(columns = {'year':'Year'}, inplace = True)
CO2Data_Cntry_GDF.rename(columns = {'year':'Year'}, inplace = True)
CO2Data_glb_GDF.rename(columns = {'year':'Year'}, inplace = True)

In [17]:
#Output to csv folder
CO2Data_Full.to_csv("..\data\CO2_RawData.csv", index = False)  ## Entire Raw DAta with stations

CO2Data_Cntry_GDF.to_csv("..\data\CO2_BYCountry.csv", index = False) ## Grouped by Country

CO2Data_glb_GDF.to_csv("..\data\CO2_GlobalSummary.csv", index = False) ## Values rolled up to global level